In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import re
from datetime import datetime
import pandas as pd

from echolalia.parser import WhatsAppParser
from echolalia.contextualizer import Contextualizer

IndentationError: expected an indented block after 'elif' statement on line 100 (parser.py, line 103)

In [3]:
parser = WhatsAppParser()

S3_BUCKET_NAME = "smcphers-echolalia"
CHAT_LOG_FILENAME = "data/_chat.txt"

# Parse the chat log into dicts and then into a DataFrame
messages = parser.parse_chat_log(bucket=S3_BUCKET_NAME, chat_log_filename=CHAT_LOG_FILENAME)

NameError: name 'WhatsAppParser' is not defined

In [ ]:
# Begin to contextualize
# Sort by timestamp
messages = messages.sort_values(by='timestamp')

# Add time between last messages
messages['time_diff'] = messages['timestamp'].diff()

In [8]:
# Create a "group" whenever the user changes
messages['group'] = (messages['user'] != messages['user'].shift()).cumsum()
messages['num_messages'] = None

In [9]:
messages

,timestamp,user,message,exception,chatline,time_diff,group,num_messages
0,2022-01-17 23:15:56,Cat,‎Messages and calls are end-to-end encrypted. ...,None,"[1/17/22, 11:15:56 PM] Cat: ‎Messages and call...",NaT,1,None
1,2022-01-17 23:15:56,Selwyn-Lloyd McPherson,Hello?!,None,"[1/17/22, 11:15:56 PM] Selwyn-Lloyd McPherson:...",0 days 00:00:00,2,None
2,2022-01-17 23:16:26,Cat,HI,None,"[1/17/22, 11:16:26 PM] Cat: HI",0 days 00:00:30,3,None
3,2022-01-17 23:17:24,Cat,oh good i do have it on my computer already,None,"[1/17/22, 11:17:24 PM] Cat: oh good i do have ...",0 days 00:00:58,3,None
4,2022-01-17 23:17:24,Selwyn-Lloyd McPherson,"Success!\n‎[1/17/22, 11:19:47 PM] Selwyn-Lloyd...",None,"[1/17/22, 11:17:24 PM] Selwyn-Lloyd McPherson:...",0 days 00:00:00,4,None
...,...,...,...,...,...,...,...,...
53904,2024-08-06 12:44:43,Cat,Cough cough wheezeeeee cough,None,"[8/6/24, 12:44:43 PM] Cat: Cough cough wheezee...",1 days 19:38:08,27967,None
53905,2024-08-08 13:13:14,Cat,My turn in the er,None,"[8/8/24, 1:13:14 PM] Cat: My turn in the er",2 days 00:28:31,27967,None
53906,2024-08-08 16:41:41,Cat,Ya girls got cirrhosis,None,"[8/8/24, 4:41:41 PM] Cat: Ya girls got cirrhosis",0 days 03:28:27,27967,None
53907,2024-08-08 16:41:49,Cat,Among other things,None,"[8/8/24, 4:41:49 PM] Cat: Among other things",0 days 00:00:08,27967,None


In [121]:
import numpy as np

# Group by this new "group" column and concatenate the values in "messages"
messages_combined = messages.groupby('group', as_index=False).agg({

    'user': 'first',            # Take the first value of 'user' for each group
    'timestamp': 'first',       # Take the first value of 'timestamp' for each group
    'message': ' '.join,        # Concatenate the values of 'message'
    'num_messages': 'size',     # Count the number of messages
    'time_diff': np.median,     # Take the median value of 'time_diff'
    'chatline': '. '.join       # Concatenate the values of 'chatline'
})
messages_combined

KeyError: 'group'

In [112]:
messages_combined[:20]

,group,user,timestamp,message,num_messages,time_diff,chatline
0,1,Cat,2022-01-17 23:15:56,‎Messages and calls are end-to-end encrypted. ...,1,NaT,"[1/17/22, 11:15:56 PM] Cat: ‎Messages and call..."
1,2,Selwyn-Lloyd McPherson,2022-01-17 23:15:56,Hello?!,1,0 days 00:00:00,"[1/17/22, 11:15:56 PM] Selwyn-Lloyd McPherson:..."
2,3,Cat,2022-01-17 23:16:26,HI oh good i do have it on my computer already,2,0 days 00:00:44,"[1/17/22, 11:16:26 PM] Cat: HI. [1/17/22, 11:1..."
3,4,Selwyn-Lloyd McPherson,2022-01-17 23:17:24,"Success!\n‎[1/17/22, 11:19:47 PM] Selwyn-Lloyd...",2,0 days 00:01:16,"[1/17/22, 11:17:24 PM] Selwyn-Lloyd McPherson:..."
4,5,Cat,2022-01-17 23:20:17,yes! https://photobucket.com/u/intercat can yo...,2,0 days 00:00:22,"[1/17/22, 11:20:17 PM] Cat: yes!. [1/17/22, 11..."
5,6,Selwyn-Lloyd McPherson,2022-01-17 23:21:12,Oh! Haha the image that comes up is so random,2,0 days 00:00:19,"[1/17/22, 11:21:12 PM] Selwyn-Lloyd McPherson:..."
6,7,Cat,2022-01-17 23:21:37,i know lol a friend of mine made me a header f...,1,0 days 00:00:19,"[1/17/22, 11:21:37 PM] Cat: i know lol a frien..."
7,8,Selwyn-Lloyd McPherson,2022-01-17 23:21:55,This is a weird format,1,0 days 00:00:18,"[1/17/22, 11:21:55 PM] Selwyn-Lloyd McPherson:..."
8,9,Cat,2022-01-17 23:22:16,https://photobucket.com/u/intercat/a/191f5bea-...,3,0 days 00:00:21,"[1/17/22, 11:22:16 PM] Cat: https://photobucke..."
9,10,Selwyn-Lloyd McPherson,2022-01-17 23:23:30,Decaying abandoned! How did you know!?,1,0 days 00:00:02,"[1/17/22, 11:23:30 PM] Selwyn-Lloyd McPherson:..."


In [2]:
import re
import pandas as pd
from datetime import datetime

lines = """
[1/17/22, 11:16:26/ue202fPM] Cat: HI
/ue200e[1/17/22, 11:19:47/ue202fPM] Selwyn-Lloyd McPherson: /ue200e<attached: 00000006-PHOTO-2022-01-17-23-19-47.jpg> {capture timestamp}
[1/17/22, 11:16:26/ue202fPM] Cat: HI
[1/15/24, 7:48:22/ue202fPM] Selwyn-Lloyd McPherson: /ue200eVoice call, /ue200eNo answer
[1/15/24, 7:48:28/ue202fPM] Selwyn-Lloyd McPherson: /ue200eVideo call, /ue200e5 min
[1/17/22, 11:22:16/ue202fPM] Cat: https://photobucket.com/u/intercat/a/191f5bea-e215-4689-a3eb-4972c95808c3

https://photobucket.com/u/intercat/a/36e7ca4a-bd79-432d-8592-1f43492dbc3e. I love this
I love this, too
[1/28/24, 4:19:06/ue202fAM] Selwyn-Lloyd McPherson: Also I would only edit it when I felt drunk and pithy /ue200e<This message was edited>
[1/28/24, 4:19:06/ue202fAM] Selwyn-Lloyd McPherson: Another thing
[1/10/24, 1:24:32/ue202fPM] Cat: /ue200eMissed voice call, /ue200eTap to call back
[1/1/23, 11:30:08/ue202fPM] Selwyn-Lloyd McPherson: /ue200eYou deleted this message.
hahaha amazing typo ❤️
[1/16/24, 1:27:14/ue202fAM] Cat: /ue200eVoice call, /ue200e5 min
[1/16/24, 1:27:15/ue202fAM] Cat: Dumb voice call!
"""

In [3]:
def _sanitize_message(message) -> str:
    """
    Sanitize a line of text from a WhatsApp chat log by removing special characters
    """
    # Individual throaway cases
    if "/ue200e<attached:" in message:
        message = ""
    elif "/ue200eVoice call" in message:
        message = ""
    elif "/ue200eMissed voice call" in message:
        message = ""
    elif "/ue200eVideo call" in message:
        message = ""
    elif "/ue200eYou deleted this message." in message:
        message = ""
    elif "/ue200eTap to call back" in message:
        message = ""

    # Alays perform these replacements
    if " /ue200e<This message was edited>" in message:                      # If the message was edited
        message = message.replace(" /ue200e<This message was edited>", "")
    message = re.sub(r'http\S+', '', message)                               # Remove URLs
    message = message.strip()                                               # Remove leading/trailing whitespace

    return message

In [4]:
def generate_payload(line: str) -> dict:
    """
    For a given line, generate a an appropriate payload that describes the elements of the line.

    Parameters
    ----------
    line : str
        The line to parse
    
    Returns
    -------
    dict
        A dictionary containing the parsed elements of the line

    """

In [5]:
lines = """[1/17/22, 11:15:56\u202fPM] Cat: \u200eMessages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them.
[1/17/22, 11:22:16/ue202fPM] Cat: https://photobucket.com/u/intercat/a/191f5bea-e215-4689-a3eb-4972c95808c3

https://photobucket.com/u/intercat/a/36e7ca4a-bd79-432d-8592-1f43492dbc3e. I love this
I love this, too
[1/28/24, 4:19:06/ue202fAM] Selwyn-Lloyd McPherson: Also I would only edit it when I felt drunk and pithy /ue200e<This message was edited>
[1/28/24, 4:19:06/ue202fAM] Selwyn-Lloyd McPherson: Another thing
[1/10/24, 1:24:32/ue202fPM] Cat: /ue200eMissed voice call, /ue200eTap to call back
[1/1/23, 11:30:08/ue202fPM] Selwyn-Lloyd McPherson: /ue200eYou deleted this message.
hahaha amazing typo ❤️
[1/16/24, 1:27:14/ue202fAM] Cat: /ue200eVoice call, /ue200e5 min
[1/16/24, 1:27:15/ue202fAM] Cat: Dumb voice call!
"""

In [8]:
"""
All messages suggests the raw list of individual lines, assigned to users and timestamps, with exceptions listed if the message is not to be included. The 
goal is to create a 1:1 mapping of raw messages with the parsed messages, and to include the exceptions in the parsed messages. This will allow for a
more granular analysis of the messages, and will allow for the user to see the raw message that was parsed, and the parsed message that was created.
"""

all_messages = []

# This is rather bestpoke
# - Ignore any characters before the timestamp
# - Extract the timestamp
# - Extract the user (should end with the first colon)
# - Extract the message (may include colons as well)

# TODO Compile this
pattern = r'[^.]*?\[(\d{1,2}/\d{1,2}/\d{2}), (\d{1,2}:\d{2}:\d{2})\/ue202f([APM]{2})\] (.+?): (.*)'

# Maintain current user and timestamp for multi-line messages
current_user = current_timestamp = None

# Iterate over each line
for line in lines.splitlines():
    # Initialize the return payload
    payload = {
        "timestamp": None,
        "user": None,
        "message": None,
        "exception": None,
        "chatline": line
    }

    # Find regex matches, if the exist
    matches = re.match(pattern, line)

    try:
        # Extract elements from the line
        date = matches.group(1)                    # Date
        time = matches.group(2)                    # Time
        ampm = matches.group(3)                    # AM/PM
        timestamp = datetime.strptime(f"{date} {time} {ampm}", "%m/%d/%y %I:%M:%S %p") # Timestamp
        user = matches.group(4)                    # Username
        message = matches.group(5)                 # Message

        # Update the payload
        payload["user"] = current_user = user
        payload["timestamp"] = current_timestamp = timestamp
        
        # Sanitize and assign the correctly formatted payload message
        payload["message"] = _sanitize_message(message)
    except Exception as e:
        # This is perhaps one part of a multi-line message
        if current_user and current_timestamp:
            payload["user"] = current_user
            payload["timestamp"] = current_timestamp
            
            # Sanitize and assign the raw chatline
            payload["message"] = _sanitize_message(payload["chatline"])
            
            continue
    finally:
        # At this point they payload should be as complete as possible

        # If something is really off, exclude any totally disconnected mesages (i.e. no user or timestamp)
        if payload["user"] and payload["timestamp"]:
            all_messages.append(payload)
        else:
            print("[Hopefully just opening empty line, skipping: ", line)
            continue
            
        # Update exception if message is empty
        if not payload["message"]:
            payload["exception"] = "No content to message"
        
        continue
messages = pd.DataFrame(all_messages)

[Hopefully just opening empty line, skipping:  [1/17/22, 11:15:56 PM] Cat: ‎Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them.


In [9]:
messages

,timestamp,user,message,exception,chatline
0,2022-01-17 23:22:16,Cat,,No content to message,"[1/17/22, 11:22:16/ue202fPM] Cat: https://phot..."
1,2022-01-17 23:22:16,Cat,,No content to message,
2,2022-01-17 23:22:16,Cat,I love this,None,https://photobucket.com/u/intercat/a/36e7ca4a-...
3,2022-01-17 23:22:16,Cat,"I love this, too",None,"I love this, too"
4,2024-01-28 04:19:06,Selwyn-Lloyd McPherson,Also I would only edit it when I felt drunk an...,None,"[1/28/24, 4:19:06/ue202fAM] Selwyn-Lloyd McPhe..."
5,2024-01-28 04:19:06,Selwyn-Lloyd McPherson,Another thing,None,"[1/28/24, 4:19:06/ue202fAM] Selwyn-Lloyd McPhe..."
6,2024-01-10 13:24:32,Cat,,No content to message,"[1/10/24, 1:24:32/ue202fPM] Cat: /ue200eMissed..."
7,2023-01-01 23:30:08,Selwyn-Lloyd McPherson,,No content to message,"[1/1/23, 11:30:08/ue202fPM] Selwyn-Lloyd McPhe..."
8,2023-01-01 23:30:08,Selwyn-Lloyd McPherson,hahaha amazing typo ❤️,None,hahaha amazing typo ❤️
9,2024-01-16 01:27:14,Cat,,No content to message,"[1/16/24, 1:27:14/ue202fAM] Cat: /ue200eVoice ..."
